<a href="https://colab.research.google.com/github/farhananwari07/ProjectML_145-157/blob/main/ModelResNet50V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import argparse
import numpy as np
from imutils import paths
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input

plt.style.use('ggplot')

In [2]:
from keras.preprocessing.image import ImageDataGenerator
augs_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)  

In [3]:
train_dir = '/content/drive/MyDrive/Praktikum Kelompok_1/model_covid/train'
train_gen = augs_gen.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'categorical',
    shuffle = False
)

Found 684 images belonging to 2 classes.


In [4]:
val_dir ='/content/drive/MyDrive/Praktikum Kelompok_1/model_covid/validation'
val_gen = augs_gen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = False
)

Found 160 images belonging to 2 classes.


In [5]:
test_dir ='/content/drive/MyDrive/Praktikum Kelompok_1/model_covid/test'
test_gen = augs_gen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = False
)

Found 8 images belonging to 3 classes.


In [6]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [7]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = True

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)


94781440/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
del model

In [9]:
HR = model.fit(
	train_gen,
	steps_per_epoch=20,
	validation_data=(val_gen),
	validation_steps=5,
	epochs=EPOCHS)


Epoch 1/20
20/20 [==============================] - 249s 11s/step - loss: 0.8014 - accuracy: 0.4468 - val_loss: 0.7424 - val_accuracy: 0.5000
Epoch 2/20
20/20 [==============================] - 89s 4s/step - loss: 0.7200 - accuracy: 0.5258 - val_loss: 0.8026 - val_accuracy: 0.5000
Epoch 3/20
20/20 [==============================] - 81s 4s/step - loss: 0.7197 - accuracy: 0.4919 - val_loss: 0.9301 - val_accuracy: 0.5000
Epoch 4/20
20/20 [==============================] - 81s 4s/step - loss: 0.7021 - accuracy: 0.5371 - val_loss: 0.9698 - val_accuracy: 0.5000
Epoch 5/20
20/20 [==============================] - 86s 4s/step - loss: 0.6908 - accuracy: 0.5532 - val_loss: 1.2895 - val_accuracy: 0.5000
Epoch 6/20
20/20 [==============================] - 85s 4s/step - loss: 0.7038 - accuracy: 0.5141 - val_loss: 2.3396 - val_accuracy: 0.5000
Epoch 7/20
20/20 [==============================] - 81s 4s/step - loss: 0.7024 - accuracy: 0.5468 - val_loss: 0.7066 - val_accuracy: 0.5000
Epoch 8/20
20/20 [

In [11]:
!ls

drive  sample_data


In [15]:
%cd /content/drive/MyDrive/DataPrak

/content/drive/MyDrive/DataPrak


In [16]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model3.h5")
print("Saved model to disk")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Saved model to disk
